In [241]:
import pandas as pd
import numpy as np
from datetime import date

In [242]:
#Loading the data
today = str(date.today())
df_branch_service = pd.read_json("branch_service_transaction_info.json")
df_customer_transaction = pd.read_json("customer_transaction_info.json")

In [243]:
#Profiling the data
df_branch_service.head(10)

,txn_id,branch_name,service,price
0,TXN-24546,MallOfAsia,Manicure,NaN
1,TXN-14642,Starmall,HairColor,NaN
2,TXN-60295,SmallMall,FootSpa,NaN
3,TXN-60295,Starmall,FootSpa,NaN
4,TXN-60295,MayMall,FootSpa,NaN
5,TXN-60295,FrankMall,FootSpa,NaN
6,TXN-40462,Starmall,HairColor,NaN
7,TXN-40462,Megamall,HairColor,NaN
8,TXN-08102,RobinsonsMall,HairColor,NaN
9,TXN-08102,SmallMall,HairColor,NaN


In [244]:
df_customer_transaction.head(10)

,txn_id,avail_date,last_name,first_name,birthday
0,TXN-24546,2030-09-08,ORTIZ,EDUARDO,1990-07-08
1,TXN-14642,2026-05-26,NIENOW,LEA,2000-11-26
2,TXN-60295,2006-09-25,LESCH,FLETA,1993-05-22
3,TXN-60295,2006-09-25,LESCH,FLETA,1993-05-22
4,TXN-60295,2006-09-25,LESCH,FLETA,1993-05-22
5,TXN-60295,2006-09-25,LESCH,FLETA,1993-05-22
6,TXN-40462,2021-08-21,KUHN,TOD,2002-11-25
7,TXN-40462,2021-08-21,KUHN,TOD,2002-11-25
8,TXN-08102,2010-04-03,JOHNSON,MILTON,2003-07-10
9,TXN-08102,2010-04-03,JOHNSON,MILTON,2003-07-10


In [245]:
print(df_branch_service.shape)
print(df_customer_transaction.shape)

(130653, 4)
(130653, 5)


In [246]:
print(df_branch_service['txn_id'].nunique())
print(df_customer_transaction['txn_id'].nunique())

62354
62354


In [247]:
#Dropping duplicates
df_branch_service = df_branch_service.drop_duplicates(subset=['txn_id'])
df_customer_transaction = df_customer_transaction.drop_duplicates(subset=['txn_id'])

In [248]:
#confirming shape
print(df_branch_service.shape)
print(df_customer_transaction.shape)

(62354, 4)
(62354, 5)


In [249]:
#checking null values count per column
df_branch_service.isnull().sum()

txn_id             0
branch_name     8880
service            0
price          11082
dtype: int64

In [250]:
df_customer_transaction.isnull().sum()

txn_id        0
avail_date    0
last_name     0
first_name    0
birthday      0
dtype: int64

In [251]:
#fill up null values with forward fill for branch_name
df_branch_service['branch_name'].unique()

array(['MallOfAsia', 'Starmall', 'SmallMall', 'RobinsonsMall', '', None,
       'MayMall', 'FrankMall', 'N/A', 'Megamall'], dtype=object)

In [252]:
#'' and None are empty values, fill them with forward fill
# first, let '' be null
df_branch_service['branch_name'] = df_branch_service.replace('',np.nan).groupby('txn_id')['branch_name'].transform('first')
# then fill null, use forward and backward to fill
df_branch_service['branch_name'] = df_branch_service['branch_name'].ffill().bfill()

In [253]:
#confirming branch_name has no null
df_branch_service.isnull().sum()

txn_id             0
branch_name        0
service            0
price          11082
dtype: int64

In [254]:
df_branch_service['branch_name'].unique()

array(['MallOfAsia', 'Starmall', 'SmallMall', 'RobinsonsMall', 'MayMall',
       'FrankMall', 'N/A', 'Megamall'], dtype=object)

In [255]:
# group mean fill prices
df_branch_service['price'] = df_branch_service['price'].fillna(df_branch_service.groupby(['branch_name','service'])['price'].transform('mean'))

In [256]:
#confirming price has no null
df_branch_service.isnull().sum()

txn_id         0
branch_name    0
service        0
price          0
dtype: int64

In [257]:
df_branch_service.head(10)

,txn_id,branch_name,service,price
0,TXN-24546,MallOfAsia,Manicure,42.651993
1,TXN-14642,Starmall,HairColor,69.583267
2,TXN-60295,SmallMall,FootSpa,80.037329
6,TXN-40462,Starmall,HairColor,69.583267
8,TXN-08102,RobinsonsMall,HairColor,66.735226
12,TXN-64262,SmallMall,Rebond,312.646002
15,TXN-41934,RobinsonsMall,HairColor,66.735226
19,TXN-39050,Starmall,Manicure,43.364281
21,TXN-43150,Starmall,Haircut,51.769018
22,TXN-49423,RobinsonsMall,Rebond,302.828204


In [258]:
df_merged = pd.merge(df_customer_transaction, df_branch_service)

In [259]:
#profiling of merged dataframe
df_merged.isnull().sum()

txn_id         0
avail_date     0
last_name      0
first_name     0
birthday       0
branch_name    0
service        0
price          0
dtype: int64

In [260]:
df_merged.sample(20)

,txn_id,avail_date,last_name,first_name,birthday,branch_name,service,price
57214,TXN-25596,2025-05-27,Fisher,Caroline,1990-09-09,FrankMall,HairColor,88.093930
57611,TXN-53753,2011-08-10,Haley,Lulu,2004-12-02,MayMall,FootSpa,100.121230
46930,TXN-01978,2016-03-19,Emard,Randal,2002-06-23,RobinsonsMall,FootSpa,100.121230
40483,TXN-17590,2012-11-07,"Purdy,,,,,,,,",Malinda........,2005-10-16,FrankMall,Manicure,55.232400
24268,TXN-10881,2007-05-08,Ritchie,Jarrell,1996-11-04,RobinsonsMall,Manicure,55.232400
34015,TXN-06209,2021-10-24,"Haley,,,,,,,,",Estevan........,2002-08-03,MayMall,Haircut,66.123457
41429,TXN-09295,2007-09-11,"Kemmer,,,,,,,,",Lynn........,2000-04-22,Starmall,Haircut,66.123457
28990,TXN-26920,2023-12-01,Gulgowski,Norris,1996-12-16,SmallMall,Pedicure,77.987989
472,TXN-23333,2006-03-24,WISOKY,ORION,2010-06-03,Megamall,Rebond,299.487424
8414,TXN-26537,2022-03-01,SCHOEN,SUSIE,2009-05-07,MallOfAsia,HairColor,68.352993


In [261]:
#filter alphabet only in last_name and first_name
df_merged['last_name'] = df_merged['last_name'].str.replace('\W', '', regex=True)
df_merged['first_name'] = df_merged['first_name'].str.replace('\W', '', regex=True)

<>:2: SyntaxWarning: invalid escape sequence '\W'
<>:3: SyntaxWarning: invalid escape sequence '\W'
<>:2: SyntaxWarning: invalid escape sequence '\W'
<>:3: SyntaxWarning: invalid escape sequence '\W'
C:\Users\juanc\AppData\Local\Temp\ipykernel_27004\1982014538.py:2: SyntaxWarning: invalid escape sequence '\W'
  df_merged['last_name'] = df_merged['last_name'].str.replace('\W', '', regex=True)
C:\Users\juanc\AppData\Local\Temp\ipykernel_27004\1982014538.py:3: SyntaxWarning: invalid escape sequence '\W'
  df_merged['first_name'] = df_merged['first_name'].str.replace('\W', '', regex=True)


In [262]:
#uppercase for last_name and first_name
df_merged['last_name'] = df_merged['last_name'].str.upper()
df_merged['first_name'] = df_merged['first_name'].str.upper()

In [263]:
#check if no special characters and all uppercase for names
df_merged.sample(20)

,txn_id,avail_date,last_name,first_name,birthday,branch_name,service,price
24168,TXN-58359,2029-10-03,DOUGLAS,THEODORE,2008-09-02,Megamall,HairColor,88.093930
34199,TXN-48690,2022-02-14,HEANEY,JEANETTE,1991-05-05,RobinsonsMall,HairColor,88.093930
36245,TXN-15118,2012-10-14,CORKERY,MARQUISE,1996-05-27,RobinsonsMall,Haircut,66.123457
44557,TXN-64305,2029-07-07,YOST,ROBERT,1994-05-09,Starmall,Haircut,66.123457
62126,TXN-10793,2014-10-17,MOHR,LESLEY,1993-10-14,SmallMall,Rebond,400.231230
59463,TXN-63223,2014-01-05,NIKOLAUS,RAYMUNDO,1994-06-06,FrankMall,Pedicure,77.987989
22257,TXN-26897,2020-07-28,GAYLORD,GARRET,1995-07-14,MayMall,Haircut,0.000000
59213,TXN-03365,2009-11-10,HALVORSON,MARIANA,2008-01-22,RobinsonsMall,Rebond,400.231230
27621,TXN-25058,2029-08-28,TORP,ALEXANDRIA,2008-09-28,RobinsonsMall,Pedicure,77.987989
38226,TXN-38872,2006-11-10,JENKINS,RAVEN,2005-11-12,Megamall,Haircut,66.123457


In [264]:
#profiling date columns
print(df_merged['birthday'].min())
print(df_merged['birthday'].max())

print(df_merged['avail_date'].min())
print(df_merged['avail_date'].max())

print(df_merged['avail_date'].describe)
print(df_merged['birthday'].describe)

print(df_merged[(df_merged['avail_date'] <= df_merged['birthday'])])

1990-01-01
2010-12-30
2005-01-01
2030-12-30
<bound method NDFrame.describe of 0        2030-09-08
1        2026-05-26
2        2006-09-25
3        2021-08-21
4        2010-04-03
            ...    
62349    2022-11-21
62350    2020-04-14
62351    2030-01-26
62352    2021-02-27
62353    2025-01-05
Name: avail_date, Length: 62354, dtype: object>
<bound method NDFrame.describe of 0        1990-07-08
1        2000-11-26
2        1993-05-22
3        2002-11-25
4        2003-07-10
            ...    
62349    1997-03-06
62350    1996-05-28
62351    1996-05-28
62352    1995-03-29
62353    1993-01-10
Name: birthday, Length: 62354, dtype: object>
          txn_id  avail_date  last_name first_name    birthday    branch_name  \
9      TXN-49423  2008-08-13      BERGE    RASHEED  2010-02-05  RobinsonsMall   
17     TXN-35244  2005-11-23     HAMMES     EUNICE  2006-06-13      SmallMall   
97     TXN-23898  2005-05-28   BOTSFORD     GIANNI  2009-03-30  RobinsonsMall   
101    TXN-17974  2008-04-12  

In [265]:
#data type of birthday and avail_date should be datetime instead of object
df_merged['avail_date'] = pd.to_datetime(df_merged['avail_date'], format='%Y-%m-%d')
df_merged['birthday'] = pd.to_datetime(df_merged['birthday'], format='%Y-%m-%d')

#confirming
print(df_merged['avail_date'].describe)
print(df_merged['birthday'].describe)

<bound method NDFrame.describe of 0       2030-09-08
1       2026-05-26
2       2006-09-25
3       2021-08-21
4       2010-04-03
           ...    
62349   2022-11-21
62350   2020-04-14
62351   2030-01-26
62352   2021-02-27
62353   2025-01-05
Name: avail_date, Length: 62354, dtype: datetime64[ns]>
<bound method NDFrame.describe of 0       1990-07-08
1       2000-11-26
2       1993-05-22
3       2002-11-25
4       2003-07-10
           ...    
62349   1997-03-06
62350   1996-05-28
62351   1996-05-28
62352   1995-03-29
62353   1993-01-10
Name: birthday, Length: 62354, dtype: datetime64[ns]>


In [266]:
#removing rows with later date than current date
df_merged = df_merged[(df_merged['avail_date'] <= today) & (df_merged['birthday'] <= today)]

In [267]:
#removing rows when avail_date happens before birthday
df_merged = df_merged[(df_merged['avail_date'] > df_merged['birthday'])]

In [268]:
#confirming
print(df_merged['birthday'].max())
print(df_merged['avail_date'].max())

print(df_merged[(df_merged['avail_date'] <= df_merged['birthday'])])

2010-12-30 00:00:00
2023-11-23 00:00:00
Empty DataFrame
Columns: [txn_id, avail_date, last_name, first_name, birthday, branch_name, service, price]
Index: []


In [269]:
df_merged.sample(20)

,txn_id,avail_date,last_name,first_name,birthday,branch_name,service,price
33865,TXN-61416,2017-04-25,ROHAN,SHANA,1992-01-18,SmallMall,HairColor,88.093930
58869,TXN-35685,2020-09-29,WOLFF,FRIDA,2001-11-10,RobinsonsMall,Pedicure,77.987989
46936,TXN-24172,2018-06-08,HALEY,NORVAL,2001-08-30,MallOfAsia,Haircut,66.123457
38905,TXN-62209,2015-06-14,JACOBSON,AILEEN,2008-04-11,RobinsonsMall,Manicure,55.232400
14515,TXN-08016,2014-12-16,ABERNATHY,RANSOM,1998-12-26,SmallMall,NailColor,0.000000
34826,TXN-59061,2014-06-05,SKILES,NAME,2008-06-14,Starmall,Rebond,400.231230
8506,TXN-53411,2018-06-30,WEST,RASHAWN,1996-08-16,MayMall,NailColor,22.716300
34294,TXN-48718,2020-11-28,BAHRINGER,DELTA,1993-07-25,Megamall,Pedicure,77.987989
41,TXN-37101,2013-12-22,RAU,JOHN,1990-07-31,MayMall,FootSpa,80.155138
46148,TXN-13447,2009-09-08,JONES,MARGIE,2007-02-27,MayMall,Haircut,66.123457


In [270]:
df_merged.shape

(43136, 8)

In [271]:
import sqlite3

In [272]:
db_connect = sqlite3.connect('datadb')

In [273]:
cur = db_connect.cursor()

In [274]:
cur.execute("create table if not exists datalab (txn_id varchar(45) not null primary key,avail_data DATE, last_name varchar(20),birthday date,branch_name varchar(30), service varchar(30), price double)")
db_connect.commit()

In [275]:
df_merged.to_sql('datalab', db_connect, if_exists='replace', index=False)

43136

In [276]:
latest_df_merged=pd.read_sql_query('select * from datalab limit 10', db_connect)

In [277]:
latest_df_merged

,txn_id,avail_date,last_name,first_name,birthday,branch_name,service,price
0,TXN-60295,2006-09-25 00:00:00,LESCH,FLETA,1993-05-22 00:00:00,SmallMall,FootSpa,80.037329
1,TXN-40462,2021-08-21 00:00:00,KUHN,TOD,2002-11-25 00:00:00,Starmall,HairColor,69.583267
2,TXN-08102,2010-04-03 00:00:00,JOHNSON,MILTON,2003-07-10 00:00:00,RobinsonsMall,HairColor,66.735226
3,TXN-64262,2018-11-23 00:00:00,HACKETT,MACI,2005-06-20 00:00:00,SmallMall,Rebond,312.646002
4,TXN-59988,2021-02-01 00:00:00,BASHIRIAN,BRICE,2004-07-20 00:00:00,RobinsonsMall,Rebond,302.828204
5,TXN-08158,2023-08-05 00:00:00,BATZ,TIMMOTHY,2008-10-06 00:00:00,MayMall,NailColor,22.716300
6,TXN-37609,2022-09-04 00:00:00,KUNDE,MARIANA,1996-10-29 00:00:00,MallOfAsia,FootSpa,78.040113
7,TXN-31294,2018-01-04 00:00:00,BREITENBERG,MADGE,2006-06-22 00:00:00,RobinsonsMall,Haircut,51.196577
8,TXN-21732,2017-03-16 00:00:00,TURCOTTE,HELLEN,1996-03-25 00:00:00,RobinsonsMall,FootSpa,75.620543
9,TXN-31732,2013-04-28 00:00:00,LOCKMAN,CHADD,2003-06-17 00:00:00,RobinsonsMall,Manicure,41.900441


In [278]:
latest_df_merged=pd.read_sql_query('select * from datalab', db_connect)

In [279]:
latest_df_merged

,txn_id,avail_date,last_name,first_name,birthday,branch_name,service,price
0,TXN-60295,2006-09-25 00:00:00,LESCH,FLETA,1993-05-22 00:00:00,SmallMall,FootSpa,80.037329
1,TXN-40462,2021-08-21 00:00:00,KUHN,TOD,2002-11-25 00:00:00,Starmall,HairColor,69.583267
2,TXN-08102,2010-04-03 00:00:00,JOHNSON,MILTON,2003-07-10 00:00:00,RobinsonsMall,HairColor,66.735226
3,TXN-64262,2018-11-23 00:00:00,HACKETT,MACI,2005-06-20 00:00:00,SmallMall,Rebond,312.646002
4,TXN-59988,2021-02-01 00:00:00,BASHIRIAN,BRICE,2004-07-20 00:00:00,RobinsonsMall,Rebond,302.828204
...,...,...,...,...,...,...,...,...
43131,TXN-42627,2009-03-28 00:00:00,GREENHOLT,MARCELLUS,1999-10-15 00:00:00,RobinsonsMall,FootSpa,100.121230
43132,TXN-17127,2012-08-19 00:00:00,CONROY,ABELARDO,1991-01-17 00:00:00,Megamall,Haircut,66.123457
43133,TXN-28428,2022-11-21 00:00:00,MULLER,LIAM,1997-03-06 00:00:00,Megamall,FootSpa,100.121230
43134,TXN-55630,2020-04-14 00:00:00,PURDY,HARMON,1996-05-28 00:00:00,FrankMall,HairColor,88.093930


In [287]:
cur.execute('CREATE VIEW IF NOT EXISTS weekly_report AS SELECT strftime(\'%W\',avail_date) AS week_of_year, service, SUM(price) AS weekly_sales FROM labex3 ORDER BY week_of_year ASC ' )